In [8]:
import json
import os
import time

from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
import gradio as gr
from openai import OpenAI
import pandas as pd

load_dotenv(dotenv_path='env')

MODEL = os.getenv('MODEL', "gpt-4o-mini")
openai_api_key = os.getenv('API_KEY')
openai_api_base = os.getenv('API_BASE', 'https://api.openai.com/v1/')

prompts = {
    "assistant_prompt": "Вы выступаете в роли опытного гида по путешествиям и музеям. Помогайте пользователю найти и понять интересную информацию о достопримечательностях, экспонатах и местах.",

    "query_prompt": '''Вы — полезный консультант по путешествиям и музеям. Вам будет представлен диалог.
Найдите в тексте последнего сообщения пользователя основную достопримечательность, экспонат или место, о котором спрашивает пользователь.
Верните поисковый запрос, отражающий интерес пользователя (ключевые слова). Если пользователь не уточняет интерес к конкретному объекту, верните краткий обобщённый запрос.
Отвечайте кратко по-русски.'''
}

SYSTEM_PROMPT = prompts["assistant_prompt"]
QUERY_PROMPT = prompts["query_prompt"]

In [5]:
# Загрузка спаршенных данных для ретривера
file_path = "/china_doc/2024_12_14 China full 02.csv"
data = pd.read_csv(file_path)
docs =  data['Clean_Text'].tolist()
docs[0]

'Трудно себе представить более загадочную, своеобразную и притягательную страну, чем Китай. Здесь есть все: горы и равнины, песчаные пляжи и курорты, знаменитые монастыри, археологические раскопки, древние города и альпинистские базы. Это третья по территории страна мира после России и Канады и первая по численности населения. Китаю принадлежит более 3400 островов, крупнейшим из которых является остров Хайнань в Южно-Китайском море. Китай – необыкновенная страна. Наука и техника здесь достигли высокого уровня развития еще несколько тысяч лет назад. Мировой известностью пользовались китайский шелк, керамические и фарфоровые изделия. Но все же особое место среди достижений китайцев принадлежит четырем великим открытиям: изобретению бумаги, книгопечатания, компаса и пороха. А Великая Китайская стена протяженностью 6 тыс. км – это поистине чудо света! Одним из наиболее востребованных китайских турпродуктов на российском рынке сегодня является пляжный отдых на острове Хайнань, но с каждым г

In [6]:
# Создание эмбедингов
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
texts = text_splitter.create_documents(docs)

vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(documents=texts)

# Функция для поиска похожих векторов
def search_relevant_info(search_query, k = 4):
    retrieved_docs = vector_store.similarity_search(search_query, k)
    return [x.page_content for x in retrieved_docs]
search_relevant_info('Какие города обычно включаются в экскурсионные туры по Китаю?')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


['китайского католицизма в Гуанчжоу является этот собор. Готический двухбашенный храм был построен в 1888 г. Здание возведено из гранита и потому выглядит монументально. Эффектно смотрятся башни, массивные купола и острые арочные верхушки.',
 'которой проходят с утра до вечера в течение недели. В Резиденции Норбулинка работает Музей Тибета. Богатая экспозиция включает уникальные книги на тибетском языке, красочные танка, музыкальные инструменты, ритуальную утварь, керамические изделия и работы народных мастеров.',
 'Городок Тунли. 18 км от Сучжоу по дороге к озеру Тайху. Небольшой городок Тунли расположен на воде. Он имеет тысячелетнюю историю и когда-то назывался Фуши. Свое современное название получил в эпоху Тан. Через его центр протекает 15 речушек. Здесь проживает около 10 тыс. жителей, и лодки для них являются более необходимым транспортным средством, чем автомобили. Главная достопримечательность города – это мосты, построенные в разные эпохи и отражающие разные архитектурные сти

In [9]:
# Объект для общения с GPT-4
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def get_completion(messages, model=MODEL, generation_config={}):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        **generation_config
    )
    return response.choices[0].message.content

# Класс, реализующий диалоговую систему
class LLMConversation:
    def __init__(self, system_prompt=SYSTEM_PROMPT, messages=None):
        self.system_prompt = system_prompt
        self.messages = messages if messages else []

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

    def get_prompt(self, relevant_info=None):
        if relevant_info and len(relevant_info) > 0:
            info_str = "Найденная релевантная информация:\n" + json.dumps(relevant_info, ensure_ascii=False)
            full_system = self.system_prompt + "\n" + info_str
        else:
            full_system = self.system_prompt
        return [{'role': 'system', 'content': full_system}] + self.messages

    def get_query(self, model):
        system_message = [{'role': 'system', 'content': QUERY_PROMPT}]
        user_message = [{'role': 'user', 'content': json.dumps(self.messages, ensure_ascii=False)}]
        config = {
            "temperature": 0.1,
            "max_tokens": 25,
        }
        return get_completion(system_message + user_message, model, config)

# Функция для обработки запроса пользователя и прокидывание информации в Gradio
def handle_user_input(user_message, chat_history, user_data):
    if 'conversation_history' not in user_data:
        user_data['conversation_history'] = []
    conv = LLMConversation(messages=user_data['conversation_history'])
    conv.add_message('user', user_message)

    query = conv.get_query(MODEL)
    relevant_info = search_relevant_info(query)

    full_prompt = conv.get_prompt(relevant_info)
    assistant_config = {"temperature": 0.4, "max_tokens": 200}
    assistant_answer = get_completion(full_prompt, MODEL, assistant_config)
    conv.add_message('assistant', assistant_answer)

    user_data['conversation_history'] = conv.messages

    return "", chat_history + [(user_message, assistant_answer)], user_data, json.dumps(relevant_info, ensure_ascii=False), query

In [10]:
# Запуск Gradio интерфейса
with gr.Blocks(title=MODEL) as app:
    user_data = gr.State(value={})
    chat_area = gr.Chatbot()
    input_area = gr.Textbox(label="Ваш запрос")
    with gr.Accordion("Информация", open=False):
        search_query_box = gr.Textbox(label="Поисковый запрос", interactive=False, show_copy_button=True)
        info_box = gr.Textbox(label="Релевантная информация", interactive=False, show_copy_button=True)

    input_area.submit(
        handle_user_input,
        [input_area, chat_area, user_data],
        [input_area, chat_area, user_data, info_box, search_query_box],
    )

app.launch(debug=True, share=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f6a1e70d8704009e25.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f6a1e70d8704009e25.gradio.live
